In [1]:
import requests
import pandas as pd
import json
from json import JSONDecoder
from pprint import pprint
import time as T

In [2]:
# http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q=45.53,87.38&date=2008-09-08&tp=1&format=json
# above is example

In [3]:
#import cleaned NFL schedule
schedule_csv = "NFL_Schedule.csv"

schedule = pd.read_csv(schedule_csv)
schedule.head()

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets
3,2008-09-07,1:02PM,1300,12,1,New England,Patriots,Patriots
4,2008-09-07,1:02PM,1300,12,1,Pittsburgh,Steelers,Steelers


In [4]:
#import cleaned NFL Stadium Locations
locations_csv = "NFL_Locations.csv"
locations = pd.read_csv(locations_csv)
#locations

In [5]:
locations["Latitude"] = locations["Latitude"].round(decimals=3)
locations["Longitude"] = locations["Longitude"].round(decimals=3)
#locations

In [6]:
#merge schedule with locations
merge = pd.merge(left=schedule, right=locations, how="left", left_on="City", right_on="Franchise")
merge.head()

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Franchise,Latitude,Longitude
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants,New York,40.43,-74.01
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles,Philadelphia,39.57,-75.07
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets,Miami,25.46,-80.12
3,2008-09-07,1:02PM,1300,12,1,New England,Patriots,Patriots,New England,42.21,-71.04
4,2008-09-07,1:02PM,1300,12,1,Pittsburgh,Steelers,Steelers,Pittsburgh,40.26,-80.00


In [7]:
clean_df = merge[["Full Date","Time","time_conv","time_index","Week","City","Home Team","Winner","Latitude","Longitude"]]
clean_df["Starting Game Temp (F)"] = ""
clean_df["weatherDesc"] = ""
clean_df["cloudcover"] = ""
clean_df

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Latitude,Longitude,Starting Game Temp (F),weatherDesc,cloudcover
0,2008-09-04,7:08PM,1900,18,1,New York,Giants,Giants,40.43,-74.01,,,
1,2008-09-07,1:02PM,1300,12,1,Philadelphia,Eagles,Eagles,39.57,-75.07,,,
2,2008-09-07,1:02PM,1300,12,1,Miami,Dolphins,Jets,25.46,-80.12,,,
3,2008-09-07,1:02PM,1300,12,1,New England,Patriots,Patriots,42.21,-71.04,,,
4,2008-09-07,1:02PM,1300,12,1,Pittsburgh,Steelers,Steelers,40.26,-80.00,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,2015-01-17,1:06PM,1300,12,Division,Carolina,Panthers,Panthers,35.14,-80.50,,,
1733,2015-01-17,4:40PM,1600,15,Division,Denver,Broncos,Broncos,39.43,-105.01,,,
1734,2015-01-24,3:05PM,1500,14,ConfChamp,Denver,Broncos,Broncos,39.43,-105.01,,,
1735,2015-01-24,6:42PM,1800,17,ConfChamp,Carolina,Panthers,Panthers,35.14,-80.50,,,


In [8]:
#for index, row in clean_df.iterrows():
 #   try:
  #      lat = row['Latitude']
   #     lng = row["Longitude"]
    #    date = row["Full Date"]
     #   location = f"{lat},{lng}"
      #  times = row["time_index"]
       # print(location)
       # print(date)
       # print(times)
        
       # T.sleep(.5)
   # except IndexError:
    #    print("error")

In [9]:
# set up table 1 df
table_4 = clean_df.iloc[339:800,:]
table_4

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Latitude,Longitude,Starting Game Temp (F),weatherDesc,cloudcover
339,2009-11-22,1:00PM,1300,12,11,Tampa Bay,Buccaneers,Saints,27.57,-82.27,,,
340,2009-11-22,1:02PM,1300,12,11,Baltimore,Ravens,Colts,39.17,-76.37,,,
341,2009-11-22,1:02PM,1300,12,11,Green Bay,Packers,Packers,44.30,-88.01,,,
342,2009-11-22,1:03PM,1300,12,11,Jacksonville,Jaguars,Jaguars,30.20,-81.40,,,
343,2009-11-22,1:03PM,1300,12,11,New York,Giants,Giants,40.43,-74.01,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,2011-12-04,1:02PM,1300,12,13,Pittsburgh,Steelers,Steelers,40.26,-80.00,,,
796,2011-12-04,1:02PM,1300,12,13,Tampa Bay,Buccaneers,Panthers,27.57,-82.27,,,
797,2011-12-04,1:02PM,1300,12,13,New England,Patriots,Patriots,42.21,-71.04,,,
798,2011-12-04,1:03PM,1300,12,13,Miami,Dolphins,Dolphins,25.46,-80.12,,,


In [11]:
base_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9b6272a2d2e34f45bab144757210906&q="
for index, row in table_4.iterrows():
    try:
        lat = row['Latitude']
        lng = row["Longitude"]
        date = row["Full Date"]
        time = row["time_index"]
        location = f"{lat},{lng}"
        weather_test = f"{base_url},{location}&date={date}&tp=1&format=json"
        weather_info = requests.get(weather_test).json()
        table_4.loc[index, "Starting Game Temp (F)"] = weather_info['data']['weather'][0]['hourly'][time]['tempF']
        table_4.loc[index, "weatherDesc"] = weather_info['data']['weather'][0]['hourly'][time]['weatherDesc']
        table_4.loc[index, "cloudcover"]= weather_info['data']['weather'][0]['hourly'][time]['cloudcover']
        
        print("-----------------")
        print(table_4.iloc[[index],10:])
        print("-----------------")
        T.sleep(1)
    except (KeyError,IndexError):
        print("-----------------")
        print("error -> skipped")
        print("-----------------")
print("-----------------")
print("-----------------")
print("completed")

-----------------
    Starting Game Temp (F) weatherDesc cloudcover
678                                              
-----------------


/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


-----------------
    Starting Game Temp (F) weatherDesc cloudcover
679                                              
-----------------


/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


-----------------
    Starting Game Temp (F) weatherDesc cloudcover
680                                              
-----------------


...

-----------------
    Starting Game Temp (F) weatherDesc cloudcover
799                                              
-----------------


/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
------

-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error 

-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
-----------------
-----------------
error -> skipped
-----------------
------

In [13]:
table_4

,Full Date,Time,time_conv,time_index,Week,City,Home Team,Winner,Latitude,Longitude,Starting Game Temp (F),weatherDesc,cloudcover
339,2009-11-22,1:00PM,1300,12,11,Tampa Bay,Buccaneers,Saints,27.57,-82.27,45,{'value': 'Light rain'},100
340,2009-11-22,1:02PM,1300,12,11,Baltimore,Ravens,Colts,39.17,-76.37,68,{'value': 'Partly cloudy'},46
341,2009-11-22,1:02PM,1300,12,11,Green Bay,Packers,Packers,44.30,-88.01,53,{'value': 'Cloudy'},77
342,2009-11-22,1:03PM,1300,12,11,Jacksonville,Jaguars,Jaguars,30.20,-81.40,53,{'value': 'Cloudy'},77
343,2009-11-22,1:03PM,1300,12,11,New York,Giants,Giants,40.43,-74.01,53,{'value': 'Cloudy'},77
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,2011-12-04,1:02PM,1300,12,13,Pittsburgh,Steelers,Steelers,40.26,-80.00,38,{'value': 'Sunny'},5
796,2011-12-04,1:02PM,1300,12,13,Tampa Bay,Buccaneers,Panthers,27.57,-82.27,58,{'value': 'Sunny'},0
797,2011-12-04,1:02PM,1300,12,13,New England,Patriots,Patriots,42.21,-71.04,66,{'value': 'Sunny'},0
798,2011-12-04,1:03PM,1300,12,13,Miami,Dolphins,Dolphins,25.46,-80.12,47,{'value': 'Overcast'},91


In [15]:
table_4.to_csv("Table 4.csv",encoding="utf-8",index=False)

In [12]:
# run for table 2
table_5 = clean_df.iloc[800:1270,:]

In [ ]:
base_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9b6272a2d2e34f45bab144757210906&q="
for index, row in table_5.iterrows():
    try:
        lat = row['Latitude']
        lng = row["Longitude"]
        date = row["Full Date"]
        time = row["time_index"]
        location = f"{lat},{lng}"
        weather_test = f"{base_url},{location}&date={date}&tp=1&format=json"
        weather_info = requests.get(weather_test).json()
        table_5.loc[index, "Starting Game Temp (F)"] = weather_info['data']['weather'][0]['hourly'][time]['tempF']
        table_5.loc[index, "weatherDesc"] = weather_info['data']['weather'][0]['hourly'][time]['weatherDesc']
        table_5.loc[index, "cloudcover"]= weather_info['data']['weather'][0]['hourly'][time]['cloudcover']
        
        #print(table_5.iloc[index])
        
        T.sleep(1.5)
    except (KeyError,IndexError):
        print("error")

print("completed")

In [ ]:
table_5

In [ ]:
table_5.to_csv("Table 2.csv",encoding="utf-8",index=False)

In [ ]:
table_6 = clean_df.iloc[,:]

In [ ]:
#run for table 3
table_6.head()

In [ ]:
base_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6cd0fcd19b014508966202712210906&q="
for index, row in table_3.iterrows():
    try:
        lat = row['Latitude']
        lng = row["Longitude"]
        date = row["Full Date"]
        time = row["time_index"]
        location = f"{lat},{lng}"
        weather_test = f"{base_url},{location}&date={date}&tp=1&format=json"
        weather_info = requests.get(weather_test).json()
        table_6.loc[index, "Starting Game Temp (F)"] = weather_info['data']['weather'][0]['hourly'][time]['tempF']
        table_6.loc[index, "weatherDesc"] = weather_info['data']['weather'][0]['hourly'][time]['weatherDesc']
        table_6.loc[index, "cloudcover"]= weather_info['data']['weather'][0]['hourly'][time]['cloudcover']
        
        #print(table_6.iloc[index])
        
        T.sleep(1)
    except (KeyError,IndexError):
        print("error")

print("completed")

In [ ]:
table_6

In [ ]:
table_6.to_csv("Table 3.csv",encoding="utf-8",index=False)

In [ ]:
table_4 = clean_df.iloc[1480:,:]
table_4.head()

In [ ]:
#http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=6f2d4be584aa413f9df230512210306&q=45.53,-87.38&date=2008-09-08&tp=1&format=json

In [ ]:
# done
table_1.head()
table_2.head()
table_3.head()

In [ ]:
table_4 = clean_df.iloc[495:975,:]
table_4.head()